In [1]:
import pandas as pd
import ollama
from openai import OpenAI
import openai
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv
import anthropic 
import json
import numpy as np

load_dotenv('.env')
openai_key = os.environ.get("OPENAI_KEY")
anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
client = OpenAI(api_key=openai_key)
anthropic_client = anthropic.Anthropic()
fireworks_model="accounts/fireworks/models/mixtral-8x22b-instruct"
fireworks_key = os.environ.get("FIREWORKS_KEY")
fireworks_client = Fireworks(api_key=fireworks_key)
TEMP = 0.7
MAX_TOKENS = 700

In [2]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=MAX_TOKENS,
    temperature=TEMP,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }

    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

def get_fireworks_completion(messages, fireworks_model=fireworks_model):
    chat_completion = fireworks_client.chat.completions.create(
        model = fireworks_model,
        messages = messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMP,   
    )
    return chat_completion

def get_anthropic_completion(messages):
    chat_completion = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=MAX_TOKENS,
        temperature=TEMP,
        messages=messages
    )   
    return chat_completion

In [3]:
relationship_tactics_general = ["gaslighting", "guilt-tripping", "love bombing", "isolation", "triangulation", "silent treatment",
                        "victim playing", "rushing intimacy", "negging", "hot and cold behaviour", "threatening to leave", "withholding affection", 
                        "chronic criticism", "economic control",
                                 "passive-aggressiveness", "belittling", "constant monitoring", "ultimatums", "retaliation",
                                 "ignoring boundaries"]
workplace_tactics_no_hierarchy = ["gaslighting", "information withholding", "guilt tripping", "public shaming or criticism", 
                     "divide and conquer", "excessive flattery", "blame shifting"]
workplace_tactics_hierarchy = ["gatekeeping", "fearmongering", "excessive monitoring", "favoritism", 
                               "scapegoating", "tokenism", "manipulative reward system"]
friendship_tactics = ["peer pressure", "flattery", "favor sharking", "backhanded compliments", "exclusion", "gaslighting",
                      "guit-tripping"]

In [5]:
responses = {}
# Repeat this process for each relationship scenario
for i, entry in enumerate(workplace_tactics_hierarchy):
    print(entry)
    responses[i] = {}
    responses[i]["conversation"] = entry

    openai_messages = [
        {'role': 'system', 'content': '''You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.'''},
        {'role': 'user', 'content': f"""Create an example for a highly realistic whatsapp chat between a romatic couple (A and B). 
         The chat should contain one person being manipulative without the other person noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    openai_messages_friends = [
        {'role': 'system', 'content': '''You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.'''},
        {'role': 'user', 'content': f"""Create an example for a highly realistic whatsapp chat between two or more friends. 
         The chat should contain at least one person being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    messages = [
        {'role': 'user', 'content': f"""You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.
         Create an example for a highly realistic whatsapp chat between a romatic couple (A and B). 
         The chat should contain one person being manipulative without the other person noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    messages_friends = [
        {'role': 'user', 'content': f"""You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.
         Create an example for a highly realistic whatsapp chat between two or more friends. 
         The chat should contain at least one person being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    openai_messages_workplace = [
        {'role': 'system', 'content': '''You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.'''},
        {'role': 'user', 'content': f"""Create an example for a highly realistic Email thread between two or more colleagues. 
         The chat should contain at least one person being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    messages_workplace = [ 
        {'role': 'user', 'content': f"""You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.
         Create an example for a highly realistic realistic Email thread between two or more colleagues. 
         The chat should contain at least one person being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    openai_messages_boss = [ 
        {'role': 'system', 'content': '''You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.'''},
        {'role': 'user', 'content': f"""Create an example for a highly realistic Email thread between one or more colleagues and their boss. 
         The chat should contain the boss being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    messages_boss = [
        {'role': 'user', 'content': f"""You are an experienced psychologist and relationship and communication expert. 
         You know everything about how people communicate and can pinpoint and recognize the most subtle forms of manipulation and toxic behaviour.
         Create an example for a highly realistic realistic Email thread between one or more colleagues and their boss. 
         The chat should contain the boss being manipulative without the others noticing by using the following conversational tactic: {entry}. 
         Return only the messages and the speakers and nothing else. Do not explain the chat."""},
    ]

    anthropic_response = get_anthropic_completion(messages=messages_boss)
    gpt_4_response = get_completion(messages=openai_messages_boss)

    responses[i]["gpt_4_response"] = gpt_4_response.choices[0].message.content
    responses[i]["claude_opus_response"] = anthropic_response.content[0].text

    df = pd.DataFrame.from_dict(responses, orient='index')
    df.to_csv("generated_chats_boss.csv")

gatekeeping
fearmongering
excessive monitoring
favoritism
scapegoating
tokenism
manipulative reward system
